In [1]:
import logging
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import argparse 
from pathlib import Path


logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
log = logging.getLogger()

%config Completer.use_jedi = False # make autocompletion works in jupyter

args = argparse.Namespace()
args.data_folder = './data/bach-next-note/'
args.train_folder = args.data_folder + 'train/'
args.val_folder = args.data_folder + 'valid/'
args.test_folder = args.data_folder + 'test/'
# args.train_fraction = 0.8
args.seed = 101
args.batch_size = 32
args.epochs = 7

paths = Path(args.train_folder).glob('**/chorale_*.csv')
train_np_list = [pd.read_csv(p).values.tolist() for p in paths]
# print(len(train_np_list[0]))

paths = Path(args.val_folder).glob('**/chorale_*.csv')
val_np_list = [pd.read_csv(p).values.tolist() for p in paths]

paths = Path(args.test_folder).glob('**/chorale_*.csv')
test_np_list = [pd.read_csv(p).values.tolist() for p in paths]

print(train_np_list[0])

[[66, 61, 57, 54], [66, 61, 57, 54], [68, 61, 59, 54], [68, 61, 59, 54], [69, 66, 61, 54], [69, 66, 61, 56], [69, 66, 61, 57], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 59, 49], [68, 65, 59, 49], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 57, 50], [66, 66, 59, 50], [66, 66, 59, 50], [68, 66, 59, 50], [68, 66, 59, 50], [69, 66, 61, 49], [69, 66, 61, 49], [69, 66, 61, 47], [69, 66, 61, 47], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 59, 49], [68, 65, 59, 49], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [66, 61, 57, 42], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [73, 66, 57, 54], [71, 66, 59, 56], [71, 66, 59, 56], [71, 65, 59, 56], [71, 65, 59, 56], [69, 66, 61, 57], [69, 66, 61, 57], [69, 66, 61, 59], [69, 66, 61, 59], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 61], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [68, 65, 61, 49], [69, 66, 61, 54], [69, 66, 61, 54], [69, 66, 61, 52], [69, 66, 

 ### Baseline

In [8]:
def baseline():
    def build_dataset_baseline(chorales):
        min_len = len(min(chorales, key=len))
        print(min_len)

        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def trunc(chorale):
            return chorale[:min_len+1]

        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        dataset = dataset.map(trunc)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
    #     for d in dataset:
    #         tf.print(d)
    #         break
        return dataset


    train_dataset = build_dataset_baseline(train_np_list)
    val_dataset = build_dataset_baseline(val_np_list)
    test_dataset = build_dataset_baseline(test_np_list)
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(1, input_shape=[None, 1])
    ])
    model.compile(loss='mse',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=[keras.metrics.mean_squared_error])
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.SimpleRNN(20, return_sequences=True),
        keras.layers.SimpleRNN(1),
    ])
    model.compile(loss='mse',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=[keras.metrics.mean_squared_error])
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    
baseline()

100
128
128
Epoch 1/7
8/8 [==============================] - 1s 33ms/step - loss: 4880.2749 - mean_squared_error: 4880.2749 - val_loss: 4915.0161 - val_mean_squared_error: 4915.0161
Epoch 2/7
8/8 [==============================] - 0s 14ms/step - loss: 4880.2749 - mean_squared_error: 4880.2749 - val_loss: 4914.9492 - val_mean_squared_error: 4914.9492
Epoch 3/7
8/8 [==============================] - 0s 14ms/step - loss: 4880.2749 - mean_squared_error: 4880.2749 - val_loss: 4914.9155 - val_mean_squared_error: 4914.9155
Epoch 4/7
8/8 [==============================] - 0s 14ms/step - loss: 4880.2749 - mean_squared_error: 4880.2749 - val_loss: 4914.8994 - val_mean_squared_error: 4914.8994
Epoch 5/7
8/8 [==============================] - 0s 16ms/step - loss: 4880.2749 - mean_squared_error: 4880.2749 - val_loss: 4914.8916 - val_mean_squared_error: 4914.8916
Epoch 6/7
8/8 [==============================] - 0s 14ms/step - loss: 4880.2749 - mean_squared_error: 4880.2749 - val_loss: 4914.8877 - va

### Sparse categorical entropy

In [39]:
import sys

def min_max_notes(all_dataset_chorales=()):
    min_note = None
    max_note = None
    def _min_max_notes():
        nonlocal min_note
        nonlocal max_note
        if min_note:
            return min_note, max_note
        min_note = sys.maxsize
        max_note = -sys.maxsize - 1
        for chorales in all_dataset_chorales:
            for ch in chorales:
                ch = np.array(ch)
                min_note = ch[ch > 0].min() if ch[ch > 0].min() < min_note else min_note
                max_note = ch.max() if ch.max() > max_note else max_note
            
        return min_note, max_note

    return _min_max_notes()

min_note, max_note = min_max_notes((train_np_list, val_np_list, test_np_list))

In [46]:
def sparse_categorical_entropy():
    def build_dataset_sce(chorales):

        min_len = len(min(chorales, key=len))
        print(min_len)

        def reshape(chorale):
            chorale = tf.reshape(chorale.to_tensor(), [-1, 1])
            return chorale
        def trunc(chorale):
            return chorale[:min_len+1]
        def transform(chorale):
            return tf.where(chorale == 0, 0, chorale - min_note)
        def target(chorale):
            X = chorale[:-1]
            Y = chorale[-1]
            return X, Y

        ragged_chorales = tf.ragged.constant(chorales)
        dataset = tf.data.Dataset.from_tensor_slices(ragged_chorales)
        dataset = dataset.map(reshape)
        dataset = dataset.map(trunc)
        dataset = dataset.map(transform)
        dataset = dataset.map(target)
        dataset = dataset.batch(32)
#         for d in dataset:
#             tf.print(d)
#             break
        return dataset


    train_dataset = build_dataset_sce(train_np_list)
    val_dataset = build_dataset_sce(val_np_list)
    test_dataset = build_dataset_sce(test_np_list)
    
#     model = keras.models.Sequential([
#         keras.layers.SimpleRNN(1, input_shape=[None, 1])
#     ])
#     model.compile(loss='sparse_categorical_entropy',
#                  optimizer=keras.optimizers.Nadam(learning_rate=0.01),
#                  metrics=['accuracy'])
#     model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    
    model = keras.models.Sequential([
        keras.layers.SimpleRNN(20, return_sequences=True, input_shape=[None, 1]),
        keras.layers.SimpleRNN(20, return_sequences=True),
        keras.layers.Dense(max_note - min_note + 1)
    ])
    model.compile(loss='sparse_categorical_crossentropy',
                 optimizer=keras.optimizers.Nadam(learning_rate=0.01),
                 metrics=['accuracy'])
    model.fit(train_dataset, epochs=args.epochs, validation_data=val_dataset)
    
sparse_categorical_entropy()

100
128
128
Epoch 1/7


InvalidArgumentError:  assertion failed: [Condition x == y did not hold element-wise:] [x (sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/Shape_1:0) = ] [32 1] [y (sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/strided_slice:0) = ] [32 100]
	 [[node sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/assert_equal_1/Assert/Assert (defined at var/folders/g9/6qklj4h53bv0c1rjnffg7bmw0000gp/T/ipykernel_29225/3457709051.py:52) ]] [Op:__inference_train_function_20056]

Function call stack:
train_function
